In [ ]:
import numpy as np

In [ ]:
class Genetic_Algorithm:
    def __init__(
        self,
        n_genes,
        n_alleles,
        size_population,
    ):
        self.n_genes = n_genes
        self.n_alleles = n_alleles
        self.size_population = size_population
        self.chromosomes = []
        self.fitness = []
    
    def init_population(self):
        self.chromosomes = []